Connected to venv (Python 3.12.0)

In [1]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, StructuredTool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
import dotenv
import os
from tqdm import tqdm
from pprint import pprint
from fuzzywuzzy import process, fuzz

tqdm.pandas()

assert dotenv.load_dotenv()

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
org_id = os.getenv("OPENAI_ORG_ID")
os.environ["OPENAI_API_KEY"] = openai_api_key
openai_model = '"gpt-3.5-turbo"'
openai_model = 'gpt-3.5-turbo-1106'
openai_model = 'gpt-4-1106-preview'

In [3]:
df = pd.read_csv('jobs.csv')
df['full_text'] = df['title'] + "\n" + df['description']

In [4]:
index_cols = ['main_job_title']
other_cols = ['synonyms', 'description'] 
TAXONOMY = pd.DataFrame(columns=index_cols + other_cols).set_index(index_cols)
global TAXONOMY

description_of_taxonomy = """
    main_job_title (string): The job title that is used as the main job title in the taxonomy. Should be the singular form of the job title.
    synonyms (string): Stringed together synonyms for the job title, "##" seperated
    description (string): "Description of the job title here. Kept short and focused on keywords."
"""

In [5]:
def stringify_taxonomy_row(ix, row):
   return f"### main job title: {ix} ###\n {row.to_dict()}\n### END ###"
   
def stringify_taxonomy_rows(df):
    return "\n".join([stringify_taxonomy_row(ix, row) for ix, row in df.iterrows()])

def top_similar_descriptions(df, new_desc, top_n=3):
    scores = process.extract(new_desc, df['description'], 
                                scorer=fuzz.token_set_ratio, limit=top_n)
    hits = []
    for score in scores:
        hit = TAXONOMY[TAXONOMY['description'] == score[0]].copy()
        hit['score'] = score[1]
        hits.append(hit)
    return pd.concat(hits)

def look_up_main_job_title_in_taxonomy(main_job_title: str) -> str:
    """
    Search the taxonomy DataFrame for a job title.
    
    :param job_title: The main job title string to search for.
    :return: one or more rows from a pandas DataFrame, stringed together.
    """
    main_job_title = main_job_title.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."

    try:
        return stringify_taxonomy_rows(TAXONOMY.loc[[main_job_title], other_cols])
    except:
        return "Job title not found in taxonomy"
    
def search_for_similar_descriptions_in_taxonomy(description: str) -> str:
    """
    Search the taxonomy DataFrame for a similar description

    :param description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    """
    description = description.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."
    top_matches = top_similar_descriptions(TAXONOMY, description)
    return stringify_taxonomy_rows(top_matches)
    
def add_job_title_to_taxonomy(
        main_job_title: str,
        synonyms: str = '',
        description: str = '',
        ) -> str:
    """
    Add a row to the taxonomy with the inputted job title in the taxonomy's "main_job_title" column. Further, you can add synonyms and a description of the job title.

    :param main_job_title: The job title that will be put in the new row's column names "main_job_title"
    :param synonyms: stringed together synonyms for the job title, seperated by ','
    :param description: the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the job title was succesfully added as a new job title in the taxonomy.
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    description = description.lower()

    try:
        TAXONOMY.loc[main_job_title] = [synonyms, description]
        return "Added job title {main_job_title} to as a new main job title in the job title taxonomy"
    except Exception as e:
       return "Could not add job title to taxonomy" + str(e)
  
def add_synonym_to_job_title_in_taxonomy(
        main_job_title: str,
        synonyms: str,
        ) -> str:
    """
    Adds a synonym to a job title in the taxonomy

    :param main_job_title: The job title the synonyms will be added to
    :param synonyms: stringed together synonyms for the job title, seperated by commas
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    try:
        TAXONOMY.loc[main_job_title, "synonyms"] = TAXONOMY.loc[main_job_title, "synonyms"] + ", " + synonyms
        return f"Succesfully added synonym(s) {synonyms} to {main_job_title}"
    except Exception as e:
        return "Could not add synonym to taxonomy" + str(e)

def replace_description_of_job_title_in_taxonomy(
        main_job_title: str,
        updated_description: str,
        ) -> str:
    """
    Replaces a description of a job title with a new description.

    :param main_job_title: The job title that will be put in the new row's column named "main_job_title"
    :param updated_description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    updated_description = updated_description.lower()
    try:
        TAXONOMY.loc[main_job_title, "description"] = updated_description
        return "Previous description for '{main_job_title}' replaced by updated description"
    except Exception as e:
        return "Could not edit description in taxonomy" + str(e)
  
def delete_job_title_from_taxonomy(
        main_job_title: str,
        ) -> str:
    """
    Deletes a main job title and its associated data from the taxonomy

    :param main_job_title: The job title that will deleted from the taxonomy
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    try:
        # delete the row from the dataframe TAXONOMY
        TAXONOMY.drop(main_job_title, inplace=True)
        return "Deleted job title from taxonomy"
    except Exception as e:
        return "Could not delete job title from taxonomy" + str(e)


# Set up the tools
look_up_main_job_title_in_taxonomy_tool = Tool.from_function(
    func=look_up_main_job_title_in_taxonomy,
    name="look_up_main_job_title_in_taxonomy",
    description="Lookup a job title in the taxonomy."
)

search_for_similar_descriptions_in_taxonomy_tool = Tool.from_function(
    func=search_for_similar_descriptions_in_taxonomy,
    name="search_for_similar_descriptions_in_taxonomy",
    description="Find most similar descriptions from the taxonomy."
)

add_job_title_to_taxonomy_tool = StructuredTool.from_function(
    func=add_job_title_to_taxonomy,
    name="add_job_title_to_taxonomy",
    description="Add a job title and it's related information to the taxonomy"
)

add_synonym_to_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=add_synonym_to_job_title_in_taxonomy,
    name="add_synonym_to_job_title_in_taxonomy",
    description="Add a synonym to a job title in the taxonomy"
)

replace_description_of_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=replace_description_of_job_title_in_taxonomy,
    name="edit_description_of_job_title_in_taxonomy",
    description="Edit the description of a job title in the taxonomy"
)

delete_job_title_from_taxonomy_tool = Tool.from_function(
    func=delete_job_title_from_taxonomy,
    name="delete_job_title_from_taxonomy",
    description="Delete a job title from the taxonomy"
)

tools = [
    look_up_job_title_in_taxonomy_tool, 
    search_for_similar_descriptions_in_taxonomy_tool, 
    add_job_title_to_taxonomy_tool, 
    add_synonym_to_job_title_in_taxonomy_tool, 
    replace_description_of_job_title_in_taxonomy_tool,
    delete_job_title_from_taxonomy_tool
   ]

NameError: name 'look_up_job_title_in_taxonomy_tool' is not defined

In [6]:
def stringify_taxonomy_row(ix, row):
   return f"### main job title: {ix} ###\n {row.to_dict()}\n### END ###"
   
def stringify_taxonomy_rows(df):
    return "\n".join([stringify_taxonomy_row(ix, row) for ix, row in df.iterrows()])

def top_similar_descriptions(df, new_desc, top_n=3):
    scores = process.extract(new_desc, df['description'], 
                                scorer=fuzz.token_set_ratio, limit=top_n)
    hits = []
    for score in scores:
        hit = TAXONOMY[TAXONOMY['description'] == score[0]].copy()
        hit['score'] = score[1]
        hits.append(hit)
    return pd.concat(hits)

def look_up_main_job_title_in_taxonomy(main_job_title: str) -> str:
    """
    Search the taxonomy DataFrame for a job title.
    
    :param job_title: The main job title string to search for.
    :return: one or more rows from a pandas DataFrame, stringed together.
    """
    main_job_title = main_job_title.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."

    try:
        return stringify_taxonomy_rows(TAXONOMY.loc[[main_job_title], other_cols])
    except:
        return "Job title not found in taxonomy"
    
def search_for_similar_descriptions_in_taxonomy(description: str) -> str:
    """
    Search the taxonomy DataFrame for a similar description

    :param description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    """
    description = description.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."
    top_matches = top_similar_descriptions(TAXONOMY, description)
    return stringify_taxonomy_rows(top_matches)
    
def add_job_title_to_taxonomy(
        main_job_title: str,
        synonyms: str = '',
        description: str = '',
        ) -> str:
    """
    Add a row to the taxonomy with the inputted job title in the taxonomy's "main_job_title" column. Further, you can add synonyms and a description of the job title.

    :param main_job_title: The job title that will be put in the new row's column names "main_job_title"
    :param synonyms: stringed together synonyms for the job title, seperated by ','
    :param description: the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the job title was succesfully added as a new job title in the taxonomy.
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    description = description.lower()

    try:
        TAXONOMY.loc[main_job_title] = [synonyms, description]
        return "Added job title {main_job_title} to as a new main job title in the job title taxonomy"
    except Exception as e:
       return "Could not add job title to taxonomy" + str(e)
  
def add_synonym_to_job_title_in_taxonomy(
        main_job_title: str,
        synonyms: str,
        ) -> str:
    """
    Adds a synonym to a job title in the taxonomy

    :param main_job_title: The job title the synonyms will be added to
    :param synonyms: stringed together synonyms for the job title, seperated by commas
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    try:
        TAXONOMY.loc[main_job_title, "synonyms"] = TAXONOMY.loc[main_job_title, "synonyms"] + ", " + synonyms
        return f"Succesfully added synonym(s) {synonyms} to {main_job_title}"
    except Exception as e:
        return "Could not add synonym to taxonomy" + str(e)

def replace_description_of_job_title_in_taxonomy(
        main_job_title: str,
        updated_description: str,
        ) -> str:
    """
    Replaces a description of a job title with a new description.

    :param main_job_title: The job title that will be put in the new row's column named "main_job_title"
    :param updated_description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    updated_description = updated_description.lower()
    try:
        TAXONOMY.loc[main_job_title, "description"] = updated_description
        return "Previous description for '{main_job_title}' replaced by updated description"
    except Exception as e:
        return "Could not edit description in taxonomy" + str(e)
  
def delete_job_title_from_taxonomy(
        main_job_title: str,
        ) -> str:
    """
    Deletes a main job title and its associated data from the taxonomy

    :param main_job_title: The job title that will deleted from the taxonomy
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    try:
        # delete the row from the dataframe TAXONOMY
        TAXONOMY.drop(main_job_title, inplace=True)
        return "Deleted job title from taxonomy"
    except Exception as e:
        return "Could not delete job title from taxonomy" + str(e)


# Set up the tools
look_up_main_job_title_in_taxonomy_tool = Tool.from_function(
    func=look_up_main_job_title_in_taxonomy,
    name="look_up_main_job_title_in_taxonomy",
    description="Lookup a job title in the taxonomy."
)

search_for_similar_descriptions_in_taxonomy_tool = Tool.from_function(
    func=search_for_similar_descriptions_in_taxonomy,
    name="search_for_similar_descriptions_in_taxonomy",
    description="Find most similar descriptions from the taxonomy."
)

add_job_title_to_taxonomy_tool = StructuredTool.from_function(
    func=add_job_title_to_taxonomy,
    name="add_job_title_to_taxonomy",
    description="Add a job title and it's related information to the taxonomy"
)

add_synonym_to_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=add_synonym_to_job_title_in_taxonomy,
    name="add_synonym_to_job_title_in_taxonomy",
    description="Add a synonym to a job title in the taxonomy"
)

replace_description_of_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=replace_description_of_job_title_in_taxonomy,
    name="edit_description_of_job_title_in_taxonomy",
    description="Edit the description of a job title in the taxonomy"
)

delete_job_title_from_taxonomy_tool = Tool.from_function(
    func=delete_job_title_from_taxonomy,
    name="delete_job_title_from_taxonomy",
    description="Delete a job title from the taxonomy"
)

tools = [
    look_up_main_job_title_in_taxonomy_tool, 
    search_for_similar_descriptions_in_taxonomy_tool, 
    add_job_title_to_taxonomy_tool, 
    add_synonym_to_job_title_in_taxonomy_tool, 
    replace_description_of_job_title_in_taxonomy_tool,
    delete_job_title_from_taxonomy_tool
   ]

In [7]:
llm = ChatOpenAI(model=openai_model, temperature=0.0, organization=org_id)
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

system_prompt = f"""
You are an assistant that builds and maintains a taxonomy of job titles.

HIGH-LEVEL TASK
You take a job posting and 
1. Identify these from the job posting: 
    a. Note all explicitly stated (verbatim) job titles from the job posting that the future employee will hold. 
    b. Note all explicitly used synonyms of the job title in the job posting (could be plural forms or abbrevations, see further details below)
    c. Make a short description of the tasks, responsibilities, qualifications, tools and area of work described in the job posting.
2. Compare the job title(s) you found in the job posting to the job titles in the taxonomy (both verbatim and by comparing descriptions). It is very  important you use your tools for this.
3. Choose whether the job title from the job posting represents:
    a. A new job title, that you should add to the taxonomy, along with any synonyms you found and the description you made.
    b. A synonym of a job title already in the taxonomy, implying you should update the synonyms for this job title in the taxonomy.
    c. A job title already in the taxonomy, implying you should check if the description of the job title in the taxonomy is accurate and update it if necessary.


DEFINITIONS
Each row in the taxonomy should be a "main job title", its synonyms and a description of tasks, responsibilities, qualifications, tools and area of work described commonly related to the job title.
A "main job title" is the job title that we use to refer to the entire row in the taxonomy. We prefer non-abbreviated singular forms as main job titles.
A synonym is e.g. plural forms of the main job title, e.g. "nurses" for the main_job_title "nurse". Or abbreviations e.g. "ASD" for "Assistant Store Director". 

EXAMPLES
These are examples of work flows and how you use the different tools. Describe you thoughts at each step.

EXAMPLE 1
You are given this (shortned) job posting: "To pædagoger på 28-30 timer til KKFO'en ved Dyvekeskolen [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagoger" (note, although the job title is plural, if the singular form is not mentioned anywhere in the job posting, you should use the plural form as the main job title in the taxonomy)
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool 'look_up_main_job_title_in_taxonomy' for checking if a job title is in the taxonomy and find that "pædagoger" is not in the taxonomy.
    You then use the tool 'search_for_similar_descriptions_in_taxonomy_tool' for searching for similar descriptions. It returns three results (by default). None are similar to the description of the job posting you made.
3. You choose that "pædagoger" is a new job title that should be added to the taxonomy. You use the tool for adding a job title to the taxonomy to add "Pædagoger" as a new job title to the taxonomy, along with the synonyms "Pædagoger" and a combined description.

EXAMPLE 2
You are given this (shortned) job posting: "Adelphi is seeking a Nurse Practitioner [...]"
1. You extract the following information:
    a. Explicitly stated job title: "Nurse Practitioner"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "Nurse Practitioner" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). One has the main job title "nurse" and a description that is similar to the description you made.
3. You choose that "Nurse Practitioner" is a synonym of "nurse" and use the tool for adding a synonym to a main job title in the taxonomy to add "Nurse Practitioner" as a synonym to "nurse".

EXAMPLE 3
You are given this (shortned) job posting: "Assistant Store Director (ASD) for [...]"
1. You extract the following information:
    a. Explicitly stated job title: "Assistant Store Director"
    b. Synonyms: "ASD"
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "Assistant Store Director" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). None are similar to the description of the job posting you made.
3. You choose that "Assistant Store Director" is a new job title that should be added to the taxonomy. You use the tool for adding a job title to the taxonomy to add "Assistant Store Director" as a new job title to the taxonomy, along with the synonyms "ASD" and the description you made.

EXAMPLE 4
You are given this (shortned) job posting: "Engageret og faglig pædagog til basisteam i Haraldsgården [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagog"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "pædagog" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). One of them "pædagoger" is the plural form of the job title you found in the job posting.
3. You decide that "pædagog", as the singular form, is a better choice for the main job title in the taxonomy than "pædagoger". You use the tool for deleting a job title in the taxonomy to delete "pædagoger". You then add "pædagog" as a main job title in the taxonomy, along with the synonym "pædagoger" and a combined description.


EXAMPLE 5
You are given this (shortned) job posting: "Engageret og faglig pædagog til basisteam i Haraldsgården [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagog"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "pædagog" is already in the taxonomy. You check the provided description and find that it is accurate. You then conclude the job title is already in the taxonomy and no further action is needed.

So again:
- Find the job title(s) in the job posting
- Check if the job title(s) is in the taxonomy
- If not, add it to the taxonomy. If it is, or something similar is, decide if the current entries need updating or if the job title is already in the taxonomy and no further action is needed.
"""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            {''.join(system_prompt)}

            Here is a list of all the tools:
            {tools}
            """,
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [8]:
agent_executor.invoke({"input": df.loc[4, 'full_text']})



> Entering new AgentExecutor chain...

Invoking: `look_up_main_job_title_in_taxonomy` with `pædagog`


Taxonomy is empty, i.e. there are no job titles at all in it currently.
Invoking: `add_job_title_to_taxonomy` with `{'main_job_title': 'Pædagog', 'synonyms': 'Pædagoger', 'description': 'En pædagog i basisteamet arbejder med børn med særlige behov, herunder psykosociale vanskeligheder, ADHD, og autismespektrum udfordringer. De er ansvarlige for at skabe strukturerede og anerkendende miljøer, anvende specialpædagogiske redskaber, udarbejde handleplaner, varetage forældresamarbejde, og samarbejde med eksterne tværfaglige partnere. Pædagoger i basisteamet er en del af en større personalegruppe og deltager i fælles pædagogiske aktiviteter, læreplaner og faglig udvikling sammen med børnehaveafdelinger. De arbejder også med pædagogisk idræt, sprog, Marte Meo, natur og science. Kandidater skal være motiverede for inkluderende arbejde, have evnen til at arbejde ud fra barnets nærmeste udvik

{'input': 'Engageret og faglig pædagog til basisteam i Haraldsgården\n![](https://cdn.ofir.dk/Images/integration/kobenhavnskommunehrmanager/description/da20e8bd-8216-4653-8f57-1b89ea2b2db7.jpeg)\r\n\r\n**Brænder du får at være med til at skabe de bedste rammer for børn med\r\nsærlige behov?**  \r\nDu får en arbejdsplads, hvor vi er mange uddannede pædagoger, har høj\r\nfaglighed, sparrer en masse og reflekterer.\r\n\r\nSom arbejdsplads ser vi os selv som attraktiv, fordi vi som selvejende\r\ninstitution både har fantastiske fysiske rammer, en sund økonomi, tydelig\r\nledelse, en god normering og mange fagligt dygtige kollegaer at sparre med.\r\n\r\n**Basisteamet**  \r\nnsker du at være med til at skabe det bedste børnehaveliv for børn med særlige\r\nbehov?  \r\nEr du engageret, faglig og positiv?  \r\nSå vil denne stilling måske være noget for dig.\r\n\r\nVores målgruppe er børn med psykosociale vanskeligheder, ADHD, sansesarte børn\r\nog børn med udfordringer inden for autismespektrum

In [9]:
agent_executor.invoke({"input": df.loc[4, 'full_text']})



> Entering new AgentExecutor chain...

Invoking: `look_up_main_job_title_in_taxonomy` with `pædagog`


### main job title: pædagog ###
 {'synonyms': 'pædagoger', 'description': 'en pædagog i basisteamet arbejder med børn med særlige behov, herunder psykosociale vanskeligheder, adhd, og autismespektrum udfordringer. de er ansvarlige for at skabe strukturerede og anerkendende miljøer, anvende specialpædagogiske redskaber, udarbejde handleplaner, varetage forældresamarbejde, og samarbejde med eksterne tværfaglige partnere. pædagoger i basisteamet er en del af en større personalegruppe og deltager i fælles pædagogiske aktiviteter, læreplaner og faglig udvikling sammen med børnehaveafdelinger. de arbejder også med pædagogisk idræt, sprog, marte meo, natur og science. kandidater skal være motiverede for inkluderende arbejde, have evnen til at arbejde ud fra barnets nærmeste udviklingszone, og være i stand til at kommunikere effektivt med forældre og kollegaer.'}
### END ###The job title "p

{'input': 'Engageret og faglig pædagog til basisteam i Haraldsgården\n![](https://cdn.ofir.dk/Images/integration/kobenhavnskommunehrmanager/description/da20e8bd-8216-4653-8f57-1b89ea2b2db7.jpeg)\r\n\r\n**Brænder du får at være med til at skabe de bedste rammer for børn med\r\nsærlige behov?**  \r\nDu får en arbejdsplads, hvor vi er mange uddannede pædagoger, har høj\r\nfaglighed, sparrer en masse og reflekterer.\r\n\r\nSom arbejdsplads ser vi os selv som attraktiv, fordi vi som selvejende\r\ninstitution både har fantastiske fysiske rammer, en sund økonomi, tydelig\r\nledelse, en god normering og mange fagligt dygtige kollegaer at sparre med.\r\n\r\n**Basisteamet**  \r\nnsker du at være med til at skabe det bedste børnehaveliv for børn med særlige\r\nbehov?  \r\nEr du engageret, faglig og positiv?  \r\nSå vil denne stilling måske være noget for dig.\r\n\r\nVores målgruppe er børn med psykosociale vanskeligheder, ADHD, sansesarte børn\r\nog børn med udfordringer inden for autismespektrum

In [10]:
agent_executor.invoke({"input": df.loc[1, 'full_text']})



> Entering new AgentExecutor chain...

Invoking: `look_up_main_job_title_in_taxonomy` with `vejleder`


Job title not found in taxonomy
Invoking: `search_for_similar_descriptions_in_taxonomy` with `support school practices, create strong learning communities, pedagogical or teaching background, experience as inclusion, LKT, or AKT advisor, co-teaching, classroom management, special educational tools, guidance for school staff, capacity building, inclusive practice, documentation tasks, conflict resolution, class meetings, individual and class level guidance`


### main job title: pædagog ###
 {'synonyms': 'pædagoger', 'description': 'en pædagog i basisteamet arbejder med børn med særlige behov, herunder psykosociale vanskeligheder, adhd, og autismespektrum udfordringer. de er ansvarlige for at skabe strukturerede og anerkendende miljøer, anvende specialpædagogiske redskaber, udarbejde handleplaner, varetage forældresamarbejde, og samarbejde med eksterne tværfaglige partnere. pædagoge

{'input': "Husum skole søger en vejleder, der kan understøtte skolens praksis med at skabe stærke børne- og læringsfællesskaber for alle skolens elever\n![](https://cdn.ofir.dk/Images/integration/kobenhavnskommunehrmanager/description/eebe7b75-9816-418d-9d43-a1744f6ac595.jpeg)\r\n\r\n**Vi mangler dig, der er uddannet pædagog eller lærer og har erfaring som\r\ninklusions-, LKT- eller AKT-vejleder.** **Din vejledning til skolens personale\r\nskal understøtte vores kapacitetsopbygning i forhold til den inkluderende\r\npraksis og i forhold til praksis med at skabe stærke fællesskaber.**\r\n\r\nVi forestiller os, at din vejledning skal foregå helt ude i praksis og\r\nforventer derfor, at du er vant til blandt andet at arbejde med Co-teaching,\r\nog at du også kan vejlede personalet i klasserumsledelse, og at du kan sparre\r\nomkring specialpædagogiske værktøjer sammen med vores lærere og pædagoger.  \r\n  \r\n**Dine arbejdsopgaver bliver mangfoldige, d** **u skal blandt andet**\r\n\r\n  * I

In [11]:
agent_executor.invoke({"input": df.loc[0, 'full_text']})



> Entering new AgentExecutor chain...

Invoking: `look_up_main_job_title_in_taxonomy` with `p\u00e6dagoger`


Job title not found in taxonomy
Invoking: `search_for_similar_descriptions_in_taxonomy` with `fritidsinstitution, b\u00f8rn, aktiviteter, planl\u00e6gning, koordinering, for\u00e6ldresamtaler, skolesamarbejde`


### main job title: vejleder ###
 {'synonyms': '', 'description': 'support school practices, create strong learning communities, pedagogical or teaching background, experience as inclusion, lkt, or akt advisor, co-teaching, classroom management, special educational tools, guidance for school staff, capacity building, inclusive practice, documentation tasks, conflict resolution, class meetings, individual and class level guidance.', 'score': 31}
### END ###
### main job title: pædagog ###
 {'synonyms': 'pædagoger', 'description': 'en pædagog i basisteamet arbejder med børn med særlige behov, herunder psykosociale vanskeligheder, adhd, og autismespektrum udfordringer. d

{'input': "To pædagoger på 28-30 timer til KKFO'en ved Dyvekeskolen\n![](https://cdn.ofir.dk/Images/integration/kobenhavnskommunehrmanager/description/da20e8bd-8216-4653-8f57-1b89ea2b2db7.jpeg)\r\n\r\n**Vi er en fritidsinstitution ved Dyvekeskolen med børn i alderen 5,5-10 år, i\r\nalt har vi ca. 350 børn, som er lige flyttet i et nyt hus. Vi mangler to\r\naktive kolleger.**\r\n\r\nVi er en aktiv og dynamisk personalegruppe, der har fuld fart på i vores\r\narbejde. Vi arbejder i aktivitetsperioder.\r\n\r\n**Vi kan tilbyde dig**\r\n\r\n  * en ambitiøs og udviklingsorienteret personalegruppe med stort engagement og høj faglighed\r\n  * en dejlig børnegruppe \r\n  * en velorganiseret afdeling, der har fokus på at arbejde med udvikling og sparring \r\n  * dygtige og spændende kolleger på alle niveauer \r\n\r\n**Vi har brug for**\r\n\r\n  * en der har erfaring indenfor planlægning og daglig koordinering af arbejdsopgaver \r\n  * en der kan inspirere, motivere og gå forrest \r\n  * kan skabe

In [12]:
TAXONOMY

,synonyms,description
main_job_title,,
pædagog,pædagoger,en pædagog i basisteamet arbejder med børn med...
vejleder,,"support school practices, create strong learni..."
p\u00e6dagoger,p\u00e6dagoger,arbejde i en fritidsinstitution med b\u00f8rn ...


In [13]:
df.loc[0, 'full_text']}

SyntaxError: unmatched '}' (<ipython-input-13-5fa3b22dcfec>, line 1)

In [14]:
df.loc[0, 'full_text']

"To pædagoger på 28-30 timer til KKFO'en ved Dyvekeskolen\n![](https://cdn.ofir.dk/Images/integration/kobenhavnskommunehrmanager/description/da20e8bd-8216-4653-8f57-1b89ea2b2db7.jpeg)\r\n\r\n**Vi er en fritidsinstitution ved Dyvekeskolen med børn i alderen 5,5-10 år, i\r\nalt har vi ca. 350 børn, som er lige flyttet i et nyt hus. Vi mangler to\r\naktive kolleger.**\r\n\r\nVi er en aktiv og dynamisk personalegruppe, der har fuld fart på i vores\r\narbejde. Vi arbejder i aktivitetsperioder.\r\n\r\n**Vi kan tilbyde dig**\r\n\r\n  * en ambitiøs og udviklingsorienteret personalegruppe med stort engagement og høj faglighed\r\n  * en dejlig børnegruppe \r\n  * en velorganiseret afdeling, der har fokus på at arbejde med udvikling og sparring \r\n  * dygtige og spændende kolleger på alle niveauer \r\n\r\n**Vi har brug for**\r\n\r\n  * en der har erfaring indenfor planlægning og daglig koordinering af arbejdsopgaver \r\n  * en der kan inspirere, motivere og gå forrest \r\n  * kan skabe resultate

In [15]:
# df = pd.read_csv('jobs.csv')
# df['full_text'] = df['title'] + "\n" + df['description']


dtype_dict = {
    'job_id': 'uint32',
    'title': 'string',
    'description': 'string',
    }

df = pd.read_csv('job_postings.csv', dtype=dtype_dict, index_col='job_id', usecols=list(dtype_dict.keys()), nrows=50)
df['full_text'] = df['title'] + "\n" + df['description']

In [16]:
df

,title,description,full_text
job_id,,,
3757940104,Hearing Care Provider,Overview HearingLife is a national hearing ca...,Hearing Care Provider Overview HearingLife is...
3757940025,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville Metalcraft of Mayville ...,Shipping & Receiving Associate 2nd shift (Beav...
3757938019,"Manager, Engineering",The TSUBAKI name is synonymous with excellenc...,"Manager, Engineering The TSUBAKI name is syno..."
3757938018,Cook,descriptionTitle Looking for a great opportu...,Cook descriptionTitle Looking for a great op...
3757937095,Principal Cloud Security Architect (Remote),"Job Summary At iHerb, we are on a mission to m...",Principal Cloud Security Architect (Remote) Jo...
3757937037,Territory Manager - New Haven,"Location: Remote, CT, United States of America...",Territory Manager - New Haven Location: Remote...
3757937004,Auto Body Techncian,Company: Gerber Collision & Glass WELCOME TO ...,Auto Body Techncian Company: Gerber Collision ...
3757936167,"ACME D8- Asst Store Director (ASD) Sussex, NJ",The First Assistant Store Director is actively...,"ACME D8- Asst Store Director (ASD) Sussex, NJ ..."
3757936097,Dishwasher,"descriptionTitle $2,000 Sign-on Bonus Guaran...","Dishwasher descriptionTitle $2,000 Sign-on B..."


In [17]:
index_cols = ['main_job_title']
other_cols = ['synonyms', 'description'] 
TAXONOMY = pd.DataFrame(columns=index_cols + other_cols).set_index(index_cols)
global TAXONOMY

description_of_taxonomy = """
    main_job_title (string): The job title that is used as the main job title in the taxonomy. Should be the singular form of the job title.
    synonyms (string): Stringed together synonyms for the job title, "##" seperated
    description (string): "Description of the job title here. Kept short and focused on keywords."
"""

In [18]:
def stringify_taxonomy_row(ix, row):
   return f"### main job title: {ix} ###\n {row.to_dict()}\n### END ###"
   
def stringify_taxonomy_rows(df):
    return "\n".join([stringify_taxonomy_row(ix, row) for ix, row in df.iterrows()])

def top_similar_descriptions(df, new_desc, top_n=3):
    scores = process.extract(new_desc, df['description'], 
                                scorer=fuzz.token_set_ratio, limit=top_n)
    hits = []
    for score in scores:
        hit = TAXONOMY[TAXONOMY['description'] == score[0]].copy()
        hit['score'] = score[1]
        hits.append(hit)
    return pd.concat(hits)

def look_up_main_job_title_in_taxonomy(main_job_title: str) -> str:
    """
    Search the taxonomy DataFrame for a job title.
    
    :param job_title: The main job title string to search for.
    :return: one or more rows from a pandas DataFrame, stringed together.
    """
    main_job_title = main_job_title.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."

    try:
        return stringify_taxonomy_rows(TAXONOMY.loc[[main_job_title], other_cols])
    except:
        return "Job title not found in taxonomy"
    
def search_for_similar_descriptions_in_taxonomy(description: str) -> str:
    """
    Search the taxonomy DataFrame for a similar description

    :param description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    """
    description = description.lower()
    if TAXONOMY.empty:
       return "Taxonomy is empty, i.e. there are no job titles at all in it currently."
    top_matches = top_similar_descriptions(TAXONOMY, description)
    return stringify_taxonomy_rows(top_matches)
    
def add_job_title_to_taxonomy(
        main_job_title: str,
        synonyms: str = '',
        description: str = '',
        ) -> str:
    """
    Add a row to the taxonomy with the inputted job title in the taxonomy's "main_job_title" column. Further, you can add synonyms and a description of the job title.

    :param main_job_title: The job title that will be put in the new row's column names "main_job_title"
    :param synonyms: stringed together synonyms for the job title, seperated by ','
    :param description: the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the job title was succesfully added as a new job title in the taxonomy.
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    description = description.lower()

    try:
        TAXONOMY.loc[main_job_title] = [synonyms, description]
        return "Added job title {main_job_title} to as a new main job title in the job title taxonomy"
    except Exception as e:
       return "Could not add job title to taxonomy" + str(e)
  
def add_synonym_to_job_title_in_taxonomy(
        main_job_title: str,
        synonyms: str,
        ) -> str:
    """
    Adds a synonym to a job title in the taxonomy

    :param main_job_title: The job title the synonyms will be added to
    :param synonyms: stringed together synonyms for the job title, seperated by commas
    """

    main_job_title = main_job_title.lower()
    synonyms = synonyms.lower()
    try:
        TAXONOMY.loc[main_job_title, "synonyms"] = TAXONOMY.loc[main_job_title, "synonyms"] + ", " + synonyms
        return f"Succesfully added synonym(s) {synonyms} to {main_job_title}"
    except Exception as e:
        return "Could not add synonym to taxonomy" + str(e)

def replace_description_of_job_title_in_taxonomy(
        main_job_title: str,
        updated_description: str,
        ) -> str:
    """
    Replaces a description of a job title with a new description.

    :param main_job_title: The job title that will be put in the new row's column named "main_job_title"
    :param updated_description: stringed together keywords and key sentences (seperated with "##") that describe the job title's most common tasks, responsibilities, qualifications, tools and area of work
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    updated_description = updated_description.lower()
    try:
        TAXONOMY.loc[main_job_title, "description"] = updated_description
        return "Previous description for '{main_job_title}' replaced by updated description"
    except Exception as e:
        return "Could not edit description in taxonomy" + str(e)
  
def delete_job_title_from_taxonomy(
        main_job_title: str,
        ) -> str:
    """
    Deletes a main job title and its associated data from the taxonomy

    :param main_job_title: The job title that will deleted from the taxonomy
    :return: String describing whether the operation was a succes
    """

    main_job_title = main_job_title.lower()
    try:
        # delete the row from the dataframe TAXONOMY
        TAXONOMY.drop(main_job_title, inplace=True)
        return "Deleted job title from taxonomy"
    except Exception as e:
        return "Could not delete job title from taxonomy" + str(e)


# Set up the tools
look_up_main_job_title_in_taxonomy_tool = Tool.from_function(
    func=look_up_main_job_title_in_taxonomy,
    name="look_up_main_job_title_in_taxonomy",
    description="Lookup a job title in the taxonomy."
)

search_for_similar_descriptions_in_taxonomy_tool = Tool.from_function(
    func=search_for_similar_descriptions_in_taxonomy,
    name="search_for_similar_descriptions_in_taxonomy",
    description="Find most similar descriptions from the taxonomy."
)

add_job_title_to_taxonomy_tool = StructuredTool.from_function(
    func=add_job_title_to_taxonomy,
    name="add_job_title_to_taxonomy",
    description="Add a job title and it's related information to the taxonomy"
)

add_synonym_to_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=add_synonym_to_job_title_in_taxonomy,
    name="add_synonym_to_job_title_in_taxonomy",
    description="Add a synonym to a job title in the taxonomy"
)

replace_description_of_job_title_in_taxonomy_tool = StructuredTool.from_function(
    func=replace_description_of_job_title_in_taxonomy,
    name="edit_description_of_job_title_in_taxonomy",
    description="Edit the description of a job title in the taxonomy"
)

delete_job_title_from_taxonomy_tool = Tool.from_function(
    func=delete_job_title_from_taxonomy,
    name="delete_job_title_from_taxonomy",
    description="Delete a job title from the taxonomy"
)

tools = [
    look_up_main_job_title_in_taxonomy_tool, 
    search_for_similar_descriptions_in_taxonomy_tool, 
    add_job_title_to_taxonomy_tool, 
    add_synonym_to_job_title_in_taxonomy_tool, 
    replace_description_of_job_title_in_taxonomy_tool,
    delete_job_title_from_taxonomy_tool
   ]

In [19]:
llm = ChatOpenAI(model=openai_model, temperature=0.0, organization=org_id)
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

system_prompt = f"""
You are an assistant that builds and maintains a taxonomy of job titles.

HIGH-LEVEL TASK
You take a job posting and 
1. Identify these from the job posting: 
    a. Note all explicitly stated (verbatim) job titles from the job posting that the future employee will hold. 
    b. Note all explicitly used synonyms of the job title in the job posting (could be plural forms or abbrevations, see further details below)
    c. Make a short description of the tasks, responsibilities, qualifications, tools and area of work described in the job posting.
2. Compare the job title(s) you found in the job posting to the job titles in the taxonomy (both verbatim and by comparing descriptions). It is very  important you use your tools for this.
3. Choose whether the job title from the job posting represents:
    a. A new job title, that you should add to the taxonomy, along with any synonyms you found and the description you made.
    b. A synonym of a job title already in the taxonomy, implying you should update the synonyms for this job title in the taxonomy.
    c. A job title already in the taxonomy, implying you should check if the description of the job title in the taxonomy is accurate and update it if necessary.


DEFINITIONS
Each row in the taxonomy should be a "main job title", its synonyms and a description of tasks, responsibilities, qualifications, tools and area of work described commonly related to the job title.
A "main job title" is the job title that we use to refer to the entire row in the taxonomy. We prefer non-abbreviated singular forms as main job titles.
A synonym is e.g. plural forms of the main job title, e.g. "nurses" for the main_job_title "nurse". Or abbreviations e.g. "ASD" for "Assistant Store Director". 

EXAMPLES
These are examples of work flows and how you use the different tools. Describe you thoughts at each step.

EXAMPLE 1
You are given this (shortned) job posting: "To pædagoger på 28-30 timer til KKFO'en ved Dyvekeskolen [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagoger" (note, although the job title is plural, if the singular form is not mentioned anywhere in the job posting, you should use the plural form as the main job title in the taxonomy)
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool 'look_up_main_job_title_in_taxonomy' for checking if a job title is in the taxonomy and find that "pædagoger" is not in the taxonomy.
    You then use the tool 'search_for_similar_descriptions_in_taxonomy_tool' for searching for similar descriptions. It returns three results (by default). None are similar to the description of the job posting you made.
3. You choose that "pædagoger" is a new job title that should be added to the taxonomy. You use the tool for adding a job title to the taxonomy to add "Pædagoger" as a new job title to the taxonomy, along with the synonyms "Pædagoger" and a combined description.

EXAMPLE 2
You are given this (shortned) job posting: "Adelphi is seeking a Nurse Practitioner [...]"
1. You extract the following information:
    a. Explicitly stated job title: "Nurse Practitioner"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "Nurse Practitioner" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). One has the main job title "nurse" and a description that is similar to the description you made.
3. You choose that "Nurse Practitioner" is a synonym of "nurse" and use the tool for adding a synonym to a main job title in the taxonomy to add "Nurse Practitioner" as a synonym to "nurse".

EXAMPLE 3
You are given this (shortned) job posting: "Assistant Store Director (ASD) for [...]"
1. You extract the following information:
    a. Explicitly stated job title: "Assistant Store Director"
    b. Synonyms: "ASD"
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "Assistant Store Director" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). None are similar to the description of the job posting you made.
3. You choose that "Assistant Store Director" is a new job title that should be added to the taxonomy. You use the tool for adding a job title to the taxonomy to add "Assistant Store Director" as a new job title to the taxonomy, along with the synonyms "ASD" and the description you made.

EXAMPLE 4
You are given this (shortned) job posting: "Engageret og faglig pædagog til basisteam i Haraldsgården [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagog"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "pædagog" is not in the taxonomy. 
   You then use the tool for searching for similar descriptions. It returns three results (by default). One of them "pædagoger" is the plural form of the job title you found in the job posting.
3. You decide that "pædagog", as the singular form, is a better choice for the main job title in the taxonomy than "pædagoger". You use the tool for deleting a job title in the taxonomy to delete "pædagoger". You then add "pædagog" as a main job title in the taxonomy, along with the synonym "pædagoger" and a combined description.


EXAMPLE 5
You are given this (shortned) job posting: "Engageret og faglig pædagog til basisteam i Haraldsgården [...]"
1. You extract the following information:
    a. Explicitly stated job title: "pædagog"
    b. Synonyms: NONE
    b. Description: YOU MAKE THIS BASED ON THE POSTING
2. You use the tool for checking if a job title is in the taxonomy and find that "pædagog" is already in the taxonomy. You check the provided description and find that it is accurate. You then conclude the job title is already in the taxonomy and no further action is needed.

So again:
- Find the job title(s) in the job posting
- Check if the job title(s) is in the taxonomy
- If not, add it to the taxonomy. If it is, or something similar is, decide if the current entries need updating or if the job title is already in the taxonomy and no further action is needed.
"""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            {''.join(system_prompt)}

            Here is a list of all the tools:
            {tools}
            """,
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [20]:
df.tail(10)

,title,description,full_text
job_id,,,
3757933843,Senior Insights Analyst - Financial Services team,Who We Are Escalent is an award-winning data ...,Senior Insights Analyst - Financial Services t...
3757933524,Speech-Language Pathology Assistant,Maxima Therapy Inc. is excited to announce tha...,Speech-Language Pathology Assistant Maxima The...
3757933458,Associate Veterinarian,We are looking to hire an Associate Veterinari...,Associate Veterinarian We are looking to hire ...
3757933450,Carpenter,We are actively recruiting for a qualified Car...,Carpenter We are actively recruiting for a qua...
3757933435,Sales Manager,Position Summary: Our Sales Manager has managi...,Sales Manager Position Summary: Our Sales Mana...
3757933320,Writer,The Program partners with the world’s most com...,Writer The Program partners with the world’s m...
3757933252,Sales Manager,Position Summary: Our Sales Manager has managi...,Sales Manager Position Summary: Our Sales Mana...
3757932930,"Director, Contributor Experience",About Appen Appen is a leader in AI enablemen...,"Director, Contributor Experience About Appen ..."
3757932856,Seasonal Beauty Advisor,"Overview Experience a place of energy, passio...",Seasonal Beauty Advisor Overview Experience a...


In [21]:
for ix, row in df[-10:].iterrows():
    agent_executor.invoke({"input": str(row['full_text'])})



> Entering new AgentExecutor chain...

Invoking: `look_up_main_job_title_in_taxonomy` with `Senior Insights Analyst`


Taxonomy is empty, i.e. there are no job titles at all in it currently.
Invoking: `add_job_title_to_taxonomy` with `{'main_job_title': 'Senior Insights Analyst', 'description': "Responsible for the development of insights and storytelling, data analysis, and reporting. Works closely with Project Management and Sales & Strategic Growth teams. Tasks include designing primary data collection processes, ensuring data quality, creating analytical plans, framing storylines, and creating deliverables. Requires a Bachelor's degree in a related field, experience in market analysis or market research, proficiency in data analysis software, and strong communication skills."}`


Added job title {main_job_title} to as a new main job title in the job title taxonomyThe job title "Senior Insights Analyst" from the job posting has been identified as a new job title and has been added

In [22]:
TAXONOMY

,synonyms,description
main_job_title,,
senior insights analyst,,responsible for the development of insights an...
speech-language pathologist assistant,slpa,the speech-language pathologist assistant will...
associate veterinarian,,"doctor of veterinary degree, active veterinary..."
carpenter,,"a carpenter is responsible for cutting, fabric..."
sales manager,,a sales manager oversees a team of executive c...
writer,,"develop creative copy for brand trends, unders..."
"director, contributor experience",,define end-to-end contributor experience for a...
seasonal beauty advisor,,a seasonal beauty advisor at ulta beauty is re...
estimator in training,,learn to assess damaged vehicles and prepare e...


In [24]:
TAXONOMY.loc["seasonal beauty advisor", 'description']

'a seasonal beauty advisor at ulta beauty is responsible for providing exceptional guest service, building and maintaining strong guest relationships, recommending products and services based on guest interests, promoting loyalty programs, informing guests of current promotions and events, handling sales and refunds, ensuring store cleanliness and merchandise standards, participating in inventory management, and staying informed about new products. this role requires a passion for the beauty industry, sales competency, and the ability to work well both independently and as part of a team. a high school diploma is preferred, and relevant work experience is beneficial. the position involves standing, bending, and lifting merchandise for a full-scheduled shift.'

In [25]:
TAXONOMY.loc["seasonal beauty advisor", 'description'] = 'a seasonal beauty advisor is responsible for providing exceptional guest service, building and maintaining strong guest relationships, recommending products and services based on guest interests, promoting loyalty programs, informing guests of current promotions and events, handling sales and refunds, ensuring store cleanliness and merchandise standards, participating in inventory management, and staying informed about new products. this role requires a passion for the beauty industry, sales competency, and the ability to work well both independently and as part of a team. a high school diploma is preferred, and relevant work experience is beneficial. the position involves standing, bending, and lifting merchandise for a full-scheduled shift.'

In [26]:
TAXONOMY

,synonyms,description
main_job_title,,
senior insights analyst,,responsible for the development of insights an...
speech-language pathologist assistant,slpa,the speech-language pathologist assistant will...
associate veterinarian,,"doctor of veterinary degree, active veterinary..."
carpenter,,"a carpenter is responsible for cutting, fabric..."
sales manager,,a sales manager oversees a team of executive c...
writer,,"develop creative copy for brand trends, unders..."
"director, contributor experience",,define end-to-end contributor experience for a...
seasonal beauty advisor,,a seasonal beauty advisor is responsible for p...
estimator in training,,learn to assess damaged vehicles and prepare e...


In [27]:
AgentExecutor?

Init signature:
AgentExecutor(
    *,
    memory: Optional[langchain_core.memory.BaseMemory] = None,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    agent: Union[langchain.agents.agent.BaseSingleActionAgent, langchain.agents.agent.BaseMultiActionAgent],
    tools: Sequence[langchain_core.tools.BaseTool],
    return_intermediate_steps: bool = False,
    max_iterations: Optional[int] = 15,
    max_execution_time: Optional[float] = None,
    early_stopping_method: str = 'force',
    handle_parsing_errors: Union[bool, str, Callable[[langchain_core.exceptions.OutputParserException], str]] = False,
    trim_intermediate_steps: Union[int, Callable[[List[Tuple[langchain_core.agents.AgentAction, 